In [28]:
from collections import defaultdict
import tqdm
import spacy
import csv
from pyserini.search import SimpleSearcher


In [2]:
nlp = spacy.load('en_core_web_sm')
searcher = SimpleSearcher('indexes/lucene-index-msmarco-doc')

In [29]:
querystring = {}
with open("dev/queries.docdev.tsv", encoding='utf8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [topicid, querystring_of_topicid] in tsvreader:
        querystring[topicid] = querystring_of_topicid

In [59]:
improved_metrics = defaultdict(dict)
with open('query_state_distilbert') as f:
    results = f.readlines()
    for line in results:
        x, y, z = line.split()
        improved_metrics[y][x] = float(z)

In [53]:
initial_metrics = defaultdict(dict)
with open('initial_dev_stats') as f:
    results = f.readlines()
    for line in results:
        x, y, z = line.split()
        initial_metrics[y][x] = float(z)

In [62]:
queries_to_docs = defaultdict(list)
queries_to_score = defaultdict(dict)
docs_to_text = {}
with open('results/dev-bm25-100.trec') as f:
    lines = f.readlines()
    for line in tqdm.tqdm(lines):
        query_id, _, doc_id, rank, score, _ = line.split()
        if query_id in improved_metrics:
            queries_to_docs[query_id].append(doc_id)
            queries_to_score[query_id][doc_id] = float(score)
            docs_to_text[doc_id] = searcher.doc(doc_id).contents().replace('\n', ' ')

100%|██████████| 519300/519300 [00:01<00:00, 279798.18it/s]


In [82]:
improved_queries = []
for q_id in improved_metrics:
    s1 = improved_metrics[q_id]['recip_rank']
    s2 = initial_metrics[q_id]['recip_rank']
    # print(s1)
    # print(s2)
    if s1 > s2 and s1 >= 0.1:
        if q_id != 'all':
            improved_queries.append((s2/s1, s1, s2, q_id, querystring[q_id]))
improved_queries.sort()
improved_queries

[(0.0256, 1.0, 0.0256, '36703', 'average gas prices in ga'),
 (0.0646, 0.5, 0.0323, '466252', 'nuclear definition in science'),
 (0.0833, 1.0, 0.0833, '900599', 'what team does will johnson play for'),
 (0.128, 0.1, 0.0128, '1075656', 'what is a chad'),
 (0.128, 0.2, 0.0256, '1102335', 'why do people buy cars'),
 (0.1666, 0.5, 0.0833, '1059045', 'what is cameroon most important resource'),
 (0.1667, 1.0, 0.1667, '1096911', 'average income for a flight engineer'),
 (0.16681668166816682, 0.3333, 0.0556, '419692', 'is ocd a disease?'),
 (0.2,
  1.0,
  0.2,
  '1097909',
  'how long is treatment for a vaginal yeast infection'),
 (0.3636, 0.25, 0.0909, '998591', 'where is mathura'),
 (0.40011997600479904, 0.1667, 0.0667, '934223', "what's your favorite pet"),
 (0.5,
  0.5,
  0.25,
  '1031682',
  'what is the surface area of the square pyramid?'),
 (0.5,
  0.5,
  0.25,
  '1033927',
  'what is the purpose of a conceptual model for advanced practice nursing'),
 (0.5, 0.5, 0.25, '367290', 'how t

In [85]:
queries_to_docs['466252'][0]

'D1265159'

In [87]:
searcher.doc('D614936').contents().replace('\n', ' ')

"<TEXT> http://www.ansto.gov.au/NuclearFacts/AboutNuclearScience/ About nuclear science About nuclear science What is nuclear science? Nuclear science is the study of the atomic world. In nuclear science, the word 'nuclear' means 'of or relating to or constituting the nucleus of an atom'. Atoms are the fundamental constituents of everything around us, and we ourselves are entirely composed of them. This means that nuclear science is crucial to understanding our universe, our world and ourselves at the atomic level. Discover more. Natural background radiation We are all exposed constantly to ionising radiation from a variety of natural and artificial sources. The sun is a major source of 'cosmic radiation'. Skiing at high altitudes and airline flights will increase our exposure to cosmic radiation. Many buildings also emit ionising radiation simply because the materials that were used to build them (clay bricks, granite, etc) are naturally radioactive. These are all examples of 'backgro